# Arrival Time Prediction - Performance
This notebook is the final notebook on the work on GP regression for arrival time prediction, which measures the models performance. The data used is loaded from train/test pickle files which contains un-processed data. Since we assume that the synchronisation functions is constant, they are simply loaded from file.

In [21]:
import numpy as np
import pandas as pd
import gp_gpy as gp
import datetime
import time
from IPython.display import display, clear_output

def rename(d):
    return d.rename(columns = {
        'latitude': 'lat', 
        'longitude': 'lon', 
        'journey_number': 'traj', 
        'segment_number': 'seg', 
        'speed': 'speed',
        'event': 'event',
        'timestamp': 'timestamp'
    })
    
route_n = 3
train = rename(pd.read_pickle('./203_train.p')).sort_values(['traj', 'seg', 'timestamp'])
test = rename(pd.read_pickle('./203_test.p')).sort_values(['traj', 'seg', 'timestamp'])

n_train_trajs = train.traj.unique()
n_train_trajs_to_use = 200
trajs_to_use = frozenset(np.random.choice(n_train_trajs, size=n_train_trajs_to_use, replace=False))
train = train.query('traj in @trajs_to_use')
print('Loaded', train.traj.unique().shape[0], 'trajectories')

Loaded 200 trajectories


# Pre-Process

In [22]:
def stop_compress(data):
    from math import radians, cos, sin, asin, sqrt, isnan
    import dateutil.parser
    
    def haversine(lon1, lat1, lon2, lat2):
        """
       Calculate the great circle distance between two points
       on the earth (specified in decimal degrees)
       """
        # convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
        # haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        r = 6371 # Radius of earth in kilometers
        
        return c * r
 
    def as_dict(d, compressed):
        return {
            'lat': d.lat,
            'lon': d.lon,
            'seg': d.seg,
            'speed': d.speed,
            'traj': int(d.traj),
            'timestamp': d.timestamp,
            'event': d.event,
            'compressed': compressed
        }

    # still requires double pd.to_datetime(..)
    def mean_timestamp(timestamps):
        x = timestamps.apply(dateutil.parser.parse)
        epochs = [ t.timestamp() for t in x ]
        mean_epoch = int(np.mean(epochs))
        dt = pd.to_datetime(mean_epoch, unit = 's')    
        return dt
 
    def compress(data):
        dict_data = [as_dict(x, 0) for x in data]
        if len(dict_data) == 1: 
            return dict_data[0]
   
        df = pd.DataFrame(dict_data)   
        df.speed = np.max(df.speed, 0) # data contains -1 sentinel values for missing speed
 
        df2 = df.drop(['timestamp', 'event', 'seg'], axis=1).apply(np.mean, axis=0)
        df2['timestamp'] = mean_timestamp(df['timestamp'])
        df2['timestamp'] = pd.to_datetime(df2['timestamp'])
   
        contains_entered_event = lambda df : df.event.transform(lambda e : e == 'EnteredEvent').any()
        df2['event'] = 'EnteredEvent' if contains_entered_event(df) else 'ObservedPositionEvent'
        df2['seg'] = df.seg.min() # In the case of overlapping segments we let the data belong to the first
 
        return as_dict(df2, 1)

    delta = 4e-3 # approx. 4 metres
    output = []
    buffer = [ data.iloc[0] ]
    for i, current in enumerate(data.itertuples()):
        distance = haversine(
            current.lat, 
            current.lon, 
            buffer[-1].lat, 
            buffer[-1].lon)
   
        if distance > delta:
            output.append(compress(buffer))
            buffer.clear()
   
        buffer.append(current)
        
        if i % 1000 == 0:
            clear_output(wait=True)
            display(str(datetime.datetime.now()) + ' processed row ' + str(i) + ' out of ' + str(data.shape[0]))


    if len(buffer) > 0:
        output.append(compress(buffer))

    compressed_data = pd.DataFrame(output)
    return compressed_data


In [23]:
t0 = time.time()
compressed_train = stop_compress(train)
elapsed = time.time() - t0
print("Train processed in", elapsed, "seconds")


'2018-12-12 07:27:24.366484 processed row 191000 out of 191156'

Train processed in 41.247472524642944 seconds


In [24]:
t0 = time.time()
compressed_test = stop_compress(test)
elapsed = time.time() - t0
print("Test processed in", elapsed, "seconds")

'2018-12-12 07:29:51.674837 processed row 693000 out of 693685'

Test processed in 149.80568432807922 seconds


In [25]:

from IPython.display import display, clear_output

def compute_progress(data):
    time_left = pd.DataFrame({'time_left': np.zeros(data.shape[0])})
    progress = pd.DataFrame({'progress': np.zeros(data.shape[0])})
    reverse_rows = data.iloc[::-1]
    last_stop_timestamp = reverse_rows.iloc[0].timestamp
    trajs = data.traj.unique()
    segs = data.seg.unique()

    sdata = data.sort_values(['traj', 'seg', 'timestamp'])
    cur_traj = 0
    cur_seg = 0
    tn = 0
    seg0 = 0
    segn = 0
    traj = sdata
    for i, d in enumerate(sdata.itertuples()):
        if d.traj > cur_traj:
            cur_seg = 0
            cur_traj = d.traj
            traj = sdata[sdata.traj == cur_traj]

        if d.seg > cur_seg:
            cur_seg = d.seg
            seg0 = i
            seg = traj[traj.seg == cur_seg]
            segn = seg.shape[0]
            tn = seg.iloc[-1].timestamp

        time_left.iloc[i] = (tn - d.timestamp).seconds
        progress.iloc[i]  = (i - seg0) / (segn - 1)
        
        if i % 1000 == 0:
            clear_output(wait=True)
            display(str(datetime.datetime.now()) + ' processed row ' + str(i) + ' out of ' + str(sdata.shape[0]))
    
    return pd.concat([
        data[['lat', 
              'lon', 
              'traj', 
              'seg', 
              'speed']],
        time_left,
        progress],
        axis = 1)


In [26]:
t0 = time.time()
progress_train = compute_progress(compressed_train)
dt = time.time() - t0
print("Train processed in", dt, " seconds")
progress_train.to_pickle('progress_train.pkl')
progress_train.head()

'2018-12-12 07:30:19.542147 processed row 109000 out of 109103'

Train processed in 24.451486349105835  seconds


,lat,lon,traj,seg,speed,time_left,progress
0,15.571011,58.414211,11,1,0.00,136.0,0.000000
1,15.571172,58.414173,11,1,-1.00,70.0,0.024390
2,15.571271,58.414303,11,1,2.93,55.0,0.048780
3,15.570987,58.414417,11,1,3.54,38.0,0.073171
4,15.570906,58.414429,11,1,4.26,37.0,0.097561


In [27]:
t0 = time.time()
progress_test = compute_progress(compressed_test)
dt = time.time() - t0
print("Test processed in", dt, " seconds")
progress_test.to_pickle('progress_test.pkl')
progress_test.head()

'2018-12-12 07:31:46.924308 processed row 391000 out of 391057'

Test processed in 87.38213300704956  seconds


,lat,lon,traj,seg,speed,time_left,progress
0,15.571018,58.414222,12,1,0.04,134.0,0.000000
1,15.571172,58.414158,12,1,-1.00,66.0,0.023810
2,15.571032,58.414198,12,1,2.88,64.0,0.047619
3,15.571301,58.414256,12,1,3.87,55.0,0.071429
4,15.571110,58.414398,12,1,4.61,46.0,0.095238


# Training

In [30]:
import numpy as np
import pandas as pd
import gp_gpy as gp
from time import time
from IPython.display import display, clear_output
route_n = 3
progress_train = pd.read_pickle('progress_train.pkl')
progress_train.head()

,lat,lon,traj,seg,speed,time_left,progress
0,15.571011,58.414211,11,1,0.00,136.0,0.000000
1,15.571172,58.414173,11,1,-1.00,70.0,0.024390
2,15.571271,58.414303,11,1,2.93,55.0,0.048780
3,15.570987,58.414417,11,1,3.54,38.0,0.073171
4,15.570906,58.414429,11,1,4.26,37.0,0.097561


In [28]:
import GPy

def synch_traj(synch_gp, data):
    X = data[['lat', 'lon']].values
    tau, _var = gp.predict(synch_gp, X)
    return data.assign(tau=tau.reshape(tau.shape[0]))

def train_seg_gps(seg, route_n, traj_n, seg_n, synch_gp):
    synched_seg = synch_traj(synch_gp, seg)
    n_restarts = 5
     # Learn GP to compute likelihood of new data
    likelihood_model = gp.build(
        gp.LIKELIHOOD,
        synched_seg[['tau']],
        synched_seg[['lat', 'lon']],
        route_n,
        traj_n,
        seg_n)
    #likelihood_model.model.kern.lengthscale = 0.05
    #likelihood_model.model.kern.variance = 5
    likelihood_model.model.kern.lengthscale.set_prior(GPy.priors.Gamma.from_EV(0.8, 0.2))
    likelihood_model.model.kern.variance.set_prior(GPy.priors.Gamma.from_EV(2, 1))
    likelihood_model.model.likelihood.variance.set_prior(GPy.priors.Gamma.from_EV(0.1, 0.005))
    #likelihood_model.model.likelihood.variance = 0.005
    #likelihood_model.model.likelihood.variance.fix()
    
    #likelihood_model.model.likelihood.variance.set_prior(GPy.priors.Gamma.from_EV(0.001, 0.00005))
    gp.train(likelihood_model, n_restarts)
    
    prediction_model = gp.build(
        gp.PREDICTION, 
        synched_seg[['tau']], 
        synched_seg[['time_left']], 
        route_n, 
        traj_n, 
        seg_n)    
    prediction_model.model.kern.lengthscale.set_prior(GPy.priors.Gamma.from_EV(0.8, 0.2))
    prediction_model.model.kern.variance.set_prior(GPy.priors.Gamma.from_EV(2, 1))
    prediction_model.model.likelihood.variance.set_prior(GPy.priors.Gamma.from_EV(0.1, 0.005))
    prediction_model.model.likelihood.variance = 0.005
    prediction_model.model.likelihood.variance.fix()
    
    # the variances are too different to capture with a good prior
    #likelihood_model.model.likelihood.variance.set_prior(GPy.priors.Gamma.from_EV(0.001, 0.00005))
    gp.train(prediction_model, n_restarts)
    
    return likelihood_model, prediction_model

In [29]:
def train_gps(data, route_n, synch_gps, conn):
    n_trajs = data.traj.unique()
    trajs = dict(tuple(data.groupby('traj')))
    for i, traj_n in enumerate(n_trajs):
        traj = trajs[traj_n] 
        n_segs = traj.seg.unique()
        segs = dict(tuple(traj.groupby('seg')))
        for seg_n in n_segs:
            seg = segs[seg_n]
            synch_gp = synch_gps[seg_n]
            lik_model, pred_model = train_seg_gps(seg, route_n, traj_n, seg_n, synch_gp)
            gp.save(lik_model, conn)
            gp.save(pred_model, conn)

        clear_output(wait=True)
        display(str(datetime.datetime.now()) + ': Trained ' + str(i) + ' out of ' + str(len(n_trajs)) + ' trajectories')

        #name = LIKELIHOOD_MODEL + str(route_n) + '-' + str(traj_n) + '-' + str(seg_n)
        #lik_model.model.save_model(name, compress=False)
        #loaded_model = GPy.load(name)
        #print(lik_model.model)
        #print(gp.load(LIKELIHOOD_MODEL, route_n, traj_n, seg_n).model) 
        #print(pred_model.model)

In [31]:
#progress_train = progress_train[progress_train.seg > 1]
synch_gps = {}
with gp.acquire_db_conn() as conn:
    for seg_n in progress_train.seg.unique():
        synch_gps[seg_n] = gp.load_synch(route_n, seg_n, conn)

t0 = time()
route_n = 3

train_gps(progress_train, route_n, synch_gps, conn)
dt = time() - t0
print("Train processed in", dt, " seconds")

'2018-12-12 07:41:54.030795: Trained 199 out of 200 trajectories'

Train processed in 606.3166592121124  seconds


# Load test data

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

test = pd.read_pickle('progress_test.pkl')
route_n = 3

# Load all GPs

In [4]:
import gp_gpy as gp
pred_gps = {}
lik_gps = {}
synch_gps = {}
with gp.acquire_db_conn() as conn:
    for i, seg_n in enumerate(test.seg.unique()):
        synch_gps[seg_n] = gp.load_synch(route_n, seg_n, conn)
        pred_gps[seg_n] = gp.load_trajs(gp.PREDICTION, route_n, seg_n, conn)
        lik_gps[seg_n] = gp.load_trajs(gp.PREDICTION, route_n, seg_n, conn)

        clear_output(wait=True)
        display('Loaded GPs for segment ' + str(i + 1) + ' out of ' + str(len(test.seg.unique())))
display('Done!')

'Loaded GPs for segment 11 out of 11'

'Done!'

# Make predictions

In [6]:
from collections import defaultdict
from scipy.stats import norm
from functools import reduce
import datetime
import pickle

def data_loglik(model, X, Y):
    def loglik(x, y):
        mu, sigma = gp.predict(model, x.reshape(1, 1))
        return -0.5*(y-mu)*np.linalg.inv(sigma)*(y-mu).T \
                -0.5*np.log(np.abs(sigma))
    return np.sum([loglik(x, y) for x, y in zip(X, Y)])

def synch_traj(synch_gp, data):
    X = data[['lat', 'lon']].values
    tau, _var = gp.predict(synch_gp, X)
    return data.assign(tau=tau.reshape(tau.shape[0]))

seg_sizes = [.2, .4, .6, .8]
mape = {s: defaultdict(dict) for s in seg_sizes}
mae =  {s: defaultdict(dict) for s in seg_sizes}
segs = dict(tuple(test.groupby('seg')))
for seg_i, seg_n in enumerate(segs):
    seg = segs[seg_n]    
    trajs = dict(tuple(seg.groupby('traj')))
    for traj_i, traj_n in enumerate(trajs):
        whole_traj = trajs[traj_n]

    
        for size in seg_sizes:

            # Extract just the first part of traj
            n = round(whole_traj.shape[0]*size)
            traj = whole_traj[:n]
            # Synchronise it
            synched_traj = synch_traj(synch_gps[seg_n], traj)
            
            # Compute weighted prediction models
            X = synched_traj[['tau']].values
            Y = synched_traj[['lat', 'lon']].values

            logliks = [data_loglik(lg, X, Y) for lg in lik_gps[seg_n]]
            models_with_loglik = zip(pred_gps[seg_n], logliks)

            models_with_weights = [(m, np.exp(loglik - max(logliks))) for m, loglik in models_with_loglik]
            preds_with_weights = \
                [gp.predict(m, X[-1,:].reshape(1, -1)) + (w,) \
                 for m, w in models_with_weights]

            # Compute the mixture model
            n = 8
            delta = 1/n
            sort = sorted(preds_with_weights, key=lambda mvw: mvw[0])
            xmin = max(0, np.floor(float(sort[0][0]-sort[0][1]*3)))
            xmax = np.ceil(float(sort[-1][0]+sort[-1][1]*3))
            xx = np.linspace(xmin, xmax, (xmax-xmin)*n)
            distributions = [w*norm.pdf(xx, float(mean), np.sqrt(float(var))) \
                             for mean, var, w in preds_with_weights]

            mixture = reduce(np.add, distributions)
            mixture = mixture/sum(mixture)

            #plt.plot(xx, mixture)

            # Make prediction
            mean = xmin + delta*round(reduce(lambda tot, m: tot + m[0]*m[1], enumerate(mixture), 0))
            #mode = xmin + delta*np.argmax(mixture)
            y = traj.iloc[-1].time_left
            pred = mean
            mae[size][seg_n][traj_n]  = float(np.abs(y-pred))
            mape[size][seg_n][traj_n] = float(np.abs((y-pred)/y))

        clear_output(wait=True)
        display(str(datetime.datetime.now()))
        display('Evaluting segment ' + str(seg_i + 1) + ' out of ' + str(len(segs)))
        display('Evaluated trajectory ' + str(traj_i + 1) + ' out of ' + str(len(trajs)))

with open('mape.pkl', 'wb') as handle:
    pickle.dump(mape, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('mae.pkl', 'wb') as handle:
    pickle.dump(mae, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Done!')

'2018-12-13 23:12:59.470632'

'Evaluting segment 11 out of 11'

'Evaluated trajectory 720 out of 720'

Done!


# Compute score
Percentage by perentage

In [7]:
import pickle
import pandas as pd
with open('mape.pkl', 'rb') as handle:
    mape = pickle.load(handle)

with open('mae.pkl', 'rb') as handle:
    mae = pickle.load(handle)
    
seg_sizes = [.2, .4, .6, .8]

for size in seg_sizes:
    final_mape = []
    final_mae = []
    for seg_n in segs:
        for traj_n in segs[seg_n].traj.unique():
            final_mape.append(mape[size][seg_n][traj_n])
            final_mae.append(mae[size][seg_n][traj_n])
    print('Size', size)
    print('MAPE', np.mean(final_mape))
    print('MAE', np.mean(final_mae))


Size 0.2
MAPE 0.2591408455675664
MAE 12.177209595959596
Size 0.4
MAPE 0.34102098271775305
MAE 10.723816287878789
Size 0.6
MAPE 0.6123383204883472
MAE 10.751499368686869
Size 0.8
MAPE 1.5501106915380023
MAE 13.416919191919192


In [8]:
Segment by segment

SyntaxError: invalid syntax (<ipython-input-8-9c3c1de971bd>, line 1)

In [9]:

test = pd.read_pickle('progress_test.pkl')
segs = dict(tuple(test.groupby('seg')))
for seg_n in segs:
    final_mape = []
    final_mae = []
    for size in seg_sizes:
        for traj_n in segs[seg_n].traj.unique():
            final_mape.append(mape[size][seg_n][traj_n])
            final_mae.append(mae[size][seg_n][traj_n])
    print('seg', seg_n)
    print('MAPE', np.mean(final_mape))
    print('MAE', np.mean(final_mae))


seg 1
MAPE 0.8382465181145535
MAE 12.107465277777777
seg 2
MAPE 0.9314885423978633
MAE 11.660807291666666
seg 3
MAPE 0.49683974065651226
MAE 6.899175347222222
seg 4
MAPE 1.6874164245156607
MAE 15.997916666666667
seg 5
MAPE 0.3724442174142777
MAE 6.131423611111111
seg 6
MAPE 0.3669121000466169
MAE 12.19123263888889
seg 7
MAPE 0.5440671979150623
MAE 13.235503472222222
seg 8
MAPE 1.3120066579870044
MAE 14.459592013888889
seg 9
MAPE 0.2763091506032301
MAE 22.112196180555557
seg 10
MAPE 0.3719260054822329
MAE 6.884548611111111
seg 11
MAPE 0.39952325572407593
MAE 7.761111111111111


In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm
import math

def plot_likelihood_models(models_by_loglik, synched_traj):
    
    def to_df(m, i):
        df = pd.DataFrame(gp.predict(m, X)[0], columns=['lat' ,'lon'])
        df['Similarity rank'] = i
        return df
    
 
    #print([(m[0].route_n, m[0].traj_n, m[0].seg_n) for m in models_by_loglik[:n_models_to_plot]])
    X = synched_traj[['tau']].values
    n_models_to_plot = 10
    models_to_plot = models_by_loglik[:n_models_to_plot]
    n_models = len(models_to_plot)
    dfs = [to_df(mloglik[0], i) for mloglik, i in zip(models_to_plot, range(n_models))]
    traj_df = synched_traj.copy()
    traj_df['Similarity rank'] = 0
    plt_df = pd.concat([traj_df] + dfs)
    
    sns.scatterplot(data=plt_df, x='lat', y='lon', hue='Similarity rank', ax=axs[0, 1])
    axs[0, 1].set_title('New trajectory with the most similair known trajectories')
    axs[0, 1].set_aspect('equal', 'datalim')    
    center_axs(axs[0, 1], synched_traj, ['lat', 'lon'])

    
def center_axs(ax, data, XY):
    ax.set_xlim(data[XY[0]].min()*0.99999, data[XY[0]].max()*1.00001)
    ax.set_ylim(data[XY[1]].min()*0.99999, data[XY[1]].max()*1.00001)
    
def mogp(traj, synch_gp, route_n, seg_n, plot):
    
    def data_loglik(model, X, Y):
        def loglik(x, y):
            mu, sigma = gp.predict(model, x.reshape(1, 1))
            return -0.5*(y-mu)*np.linalg.inv(sigma)*(y-mu).T \
                    -0.5*np.log(np.abs(sigma))
        return np.sum([loglik(x, y) for x, y in zip(X, Y)])
        
    def make_prediction(model, data):
        X = data[['lat', 'lon']].values.reshape(data.shape[0], 2)
        return gp.predict(model, X)

    def corresponding_pred_model(lik_model):
        return gp.load(PREDICTION_MODEL, lik_model.route_n, lik_model.traj_n, lik_model.seg_n)
        
    def weighted_models(synched_traj, models):
        """
        Returns models and their weights, weighted by the posterior predictive probability of the model
        normalised over the maximum likelihood assuming uniform model prior. 
        So the most probable model has weight w = 1.0, and the less likely models has weights 0 < w < 1.0.
        """
        X = synched_traj[['tau']].values
        Y = synched_traj[['lat', 'lon']].values
        models_with_loglik = [(m, data_loglik(m, X, Y)) for m in models]
        models_by_loglik = sorted(models_with_loglik, key=lambda mw: mw[1], reverse=True)
        max_loglik = models_by_loglik[0][1]
        
        if plot:
            plot_likelihood_models(models_by_loglik, synched_traj)
            
        return [(m, np.exp(loglik - max_loglik)) for m, loglik in models_by_loglik]
    
    synched_traj = synch_traj(synch_gp, traj)
    lik_models = gp.load_trajs(LIKELIHOOD_MODEL, route_n, seg_n)
    lik_models_with_weights = weighted_models(synched_traj, lik_models)
    pred_models_with_weights = [(corresponding_pred_model(lik_m), w) for lik_m, w in lik_models_with_weights]
    #print('Most similair model', lik_models_with_weights[0][0].route_n, lik_models_with_weights[0][0].traj_n, lik_models_with_weights[0][0].seg_n)
    
    if plot:
        pred_models_with_weights[0][0].model.plot(ax=axs[2, 1])
        axs[2, 1].set_title('Most similair prediction GP')
        
    latest_tau = synched_traj.iloc[-1][['tau']].values.reshape(1, 1)
    mean_var_weights = [gp.predict(m, latest_tau) + (w,) for m, w in pred_models_with_weights]
    return mean_var_weights

def predict(traj, synch_gp, route_n, seg_n, arrival_time=None, plot=False):
    mean_var_weights = mogp(traj, synch_gp, route_n, seg_n, plot)
    mean_var_weights = mean_var_weights #[:10] 
    if plot:
        sort = sorted(mean_var_weights, key=lambda mvw: mvw[0])
        xmin = max(0, np.floor(float(sort[0][0]-sort[0][1]*3)))
        xmax = np.ceil(float(sort[-1][0]+sort[-1][1]*3))
        x = np.linspace(xmin, xmax, 200)
        
        if plot:
            for mean, var, w in mean_var_weights:
                axs[1,0].plot(x, w*norm.pdf(x, float(mean), np.sqrt(float(var))))

            axs[1,0].set_title('Mixture of predictions')
            axs[1,0].set_xlabel('Seconds')
            axs[1,0].set_ylabel('Density')
            if arrival_time:
                axs[1,0].axvline(x=arrival_time, label='True arrival time')
                axs[1,0].legend()

    #models_by_similarity = sorted(mean_var_weights, key=lambda p: p[2], reverse=True)
    #models_by_mode = sorted(mean_var_weights, key=lambda p: p[0]*p[2], reverse=True)
    total_weight = sum([w for _, _, w in mean_var_weights])
    weighted_mean = sum([w*float(mean) for mean, var, w in mean_var_weights])/total_weight
    
    #weighted_means = [w*float(mean) for mean, var, w in mean_var_weights]
    #print(weighted_means)
    #mode_model = models_by_mode[0] #mean_var_weights[np.argmax(weighted_means)]
    
    #print(mode_model)
    #n = 200
    #distribution = np.sum([norm.pdf(n, float(mean), np.sqrt(float(var))) for mean, var, w in mean_var_weights])
   
    #print(weighted_means)
    #print(mode_model)
    return weighted_mean #float(mode_model[0])

#np.random.seed(8)
plot = True
if plot:
    fig, axs = plt.subplots(3,2)
    fig.set_figwidth(16)
    fig.set_figheight(24)
    
test = train #pd.read_pickle('test.pkl')
n_test_trajs_to_use = 38
n_test_trajs = test.traj.unique()
test = test[test.traj < n_test_trajs[n_test_trajs_to_use]]
traj_n = n_test_trajs[0]
test_traj = test[test.traj == traj_n]
seg_ix = 3
seg_n = test_traj.seg.unique()[seg_ix]
test_seg = test_traj[test_traj.seg == seg_n]
def predict_seg_with_plots(seg, route_n, seg_n, true_pred_gp=None):
    i = test_seg.index[0]+5 #np.random.randint(test_seg.index[0], test_seg.index[-1])
    seg = seg[seg.index <= i]
    synch_gp = gp.load_synch(route_n, seg_n, version=version)
    true = seg.iloc[-1].time_left
    pred = predict(seg, synch_gp, route_n, seg_n, true, plot=True)
    mae = float(np.abs(true-pred))
    mape = float(np.abs((true-pred)/true))
    print('true model', route_n, traj_n, seg_n)
    print('pred:', f"{pred:.2f}", 'true:', float(true), 'MAE:', f"{mae:.2f}", 'MAPE:', f"{mape:.2f}")
    
    sns.scatterplot(data=seg, x='lat', y='lon', ax=axs[0, 0])
    axs[0, 0].set_title('The new trajectory')
    axs[0, 0].set_aspect('equal', 'datalim')
    center_axs(axs[0, 0], seg, ['lat', 'lon'])
    
    synch_gp.model.plot(ax=axs[1,1])
    axs[1,1].set_title('Synchronisation GP for the new trajectory segment')
    if true_pred_gp:
        true_pred_gp.model.plot(ax=axs[2, 0])
        axs[2,0].set_title('The true prediction GP')
    
#print(test_seg[['lat', 'lon']])
#pred_gp = gp.load(PREDICTION_MODEL, route_n, traj_n, seg_n)
#predict_seg_with_plots(test_seg, route_n, seg_n, true_pred_gp=pred_gp)


In [105]:
from collections import defaultdict
test = pd.read_pickle('progress_test.pkl')
seg_sizes = [.2, .4, .6, .8]
n_segs = test.seg.unique()
segs = dict(tuple(test.groupby('seg')))
mape = {s: defaultdict(dict) for s in seg_sizes}
mae =  {s: defaultdict(dict) for s in seg_sizes}
for seg_n in n_segs[1:50]:
    seg = segs[seg_n]
    n_trajs = seg.traj.unique()
    trajs = dict(tuple(seg.groupby('traj')))
    for i, traj_n in enumerate(n_trajs):
        traj = trajs[traj_n]
        for size in seg_sizes:
            n = round(traj.shape[0]*size)
            traj_start = traj[:n]
            synch_gp = gp.load_synch(route_n, seg_n, version=version)
            X = traj_start[['lat', 'lon']]
            y = traj_start.iloc[-1].time_left
            pred = predict(X, synch_gp, route_n, int(seg_n), y, plot=False)
            mae[size][seg_n][traj_n]  = float(np.abs(y-pred))
            mape[size][seg_n][traj_n] = float(np.abs((y-pred)/y))
    
        clear_output(wait=True)
        display('Evaluated ' + str(i) + ' out of ' + str(len(n_trajs)) + ' trajs')
display('done')

'Evaluated 50 out of 720 trajs'

KeyboardInterrupt: 